# Step-to-Step to download datasets

### Download description files

In [1]:
!mkdir -p /tmp/dataset
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv -O /tmp/dataset/class-descriptions-boxable.csv
!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv -O /tmp/dataset/train-annotations-bbox.csv
!wget https://storage.googleapis.com/openimages/2018_04/validation/validation-annotations-bbox.csv -O /tmp/dataset/validation-annotations-bbox.csv
!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-bbox.csv -O /tmp/dataset/test-annotations-bbox.csv

--2019-05-08 14:04:54--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.28.240, 2800:3f0:4001:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.28.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘/tmp/dataset/class-descriptions-boxable.csv’

/tmp/dataset/class- 100%[===================>]  10,99K  --.-KB/s    in 0,008s  

2019-05-08 14:04:54 (1,35 MB/s) - ‘/tmp/dataset/class-descriptions-boxable.csv’ saved [11255/11255]

--2019-05-08 14:04:55--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.28.240, 2800:3f0:4001:801::2010
Connecting to storage.googleapis.com (storage.google

### Check files

In [2]:
!ls /tmp/dataset/

class-descriptions-boxable.csv	train-annotations-bbox.csv
test-annotations-bbox.csv	validation-annotations-bbox.csv


### Download script to download the dataset

In [3]:
!mkdir -p ../scripts
!wget https://raw.githubusercontent.com/spmallick/learnopencv/master/downloadOpenImages/downloadOI.py -O ../scripts/downloadOI.py

--2019-05-08 14:11:30--  https://raw.githubusercontent.com/spmallick/learnopencv/master/downloadOpenImages/downloadOI.py
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.128.133, 151.101.192.133, 151.101.0.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.128.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3555 (3,5K) [text/plain]
Saving to: ‘../scripts/downloadOI.py’

../scripts/download 100%[===================>]   3,47K  --.-KB/s    in 0s      

2019-05-08 14:11:30 (20,7 MB/s) - ‘../scripts/downloadOI.py’ saved [3555/3555]



### Filtering the data

In [4]:
#!cd /tmp/dataset && python3 downloadOI.py --classes 'Human_face,Human_arm,Human_head,Human_body,Human_hand,Human_leg,Human_foot' --mode train --depiction=0
# Importing libraries
import pandas as pd
import os

In [5]:
class_labels = pd.read_csv('/tmp/dataset/class-descriptions-boxable.csv', header=None, names=['label', 'name'])

In [6]:
class_labels.head()

,label,name
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [7]:
class_labels = class_labels[ class_labels["name"].isin(['Human arm','Human head','Human body','Human hand','Human leg','Human foot']) ]
class_labels

,label,name
176,/m/02p0tk3,Human body
213,/m/031n1,Human foot
220,/m/035r7c,Human leg
291,/m/04hgtk,Human head
502,/m/0dzf4,Human arm
572,/m/0k65p,Human hand


### Creating new annotation boxes

In [8]:
chunksize = 1000

with open('/tmp/dataset/train-annotations-bbox.csv') as file:
    for chunk in pd.read_csv(file, chunksize=chunksize):
        new_dataset = chunk[ chunk['LabelName'].isin(list(class_labels.label)) ]
        if not os.path.isfile('/tmp/dataset/new-train-annotations-bbox.csv'): # if file does not exist write header 
            new_dataset.to_csv('/tmp/dataset/new-train-annotations-bbox.csv', header=chunk.columns, index = False)
        else: # else it exists so append without writing the header
            new_dataset.to_csv('/tmp/dataset/new-train-annotations-bbox.csv', mode='a', header=False, index = False)

In [9]:
with open('/tmp/dataset/test-annotations-bbox.csv') as file:
    for chunk in pd.read_csv(file, chunksize=chunksize):
        new_dataset = chunk[ chunk['LabelName'].isin(list(class_labels.label)) ]
        if not os.path.isfile('/tmp/dataset/new-test-annotations-bbox.csv'): # if file does not exist write header 
            new_dataset.to_csv('/tmp/dataset/new-test-annotations-bbox.csv', header=chunk.columns, index = False)
        else: # else it exists so append without writing the header
            new_dataset.to_csv('/tmp/dataset/new-test-annotations-bbox.csv', mode='a', header=False, index = False)

In [10]:
with open('/tmp/dataset/validation-annotations-bbox.csv') as file:
    for chunk in pd.read_csv(file, chunksize=chunksize):
        new_dataset = chunk[ chunk['LabelName'].isin(list(class_labels.label)) ]
        if not os.path.isfile('/tmp/dataset/new-validation-annotations-bbox.csv'): # if file does not exist write header 
            new_dataset.to_csv('/tmp/dataset/new-validation-annotations-bbox.csv', header=chunk.columns, index = False)
        else: # else it exists so append without writing the header
            new_dataset.to_csv('/tmp/dataset/new-validation-annotations-bbox.csv', mode='a', header=False, index = False)

In [11]:
class_labels.to_csv('/tmp/dataset/new-class-descriptions-boxable.csv', header=False, index = False)

### Remove unnecessary files

In [12]:
!rm /tmp/dataset/train-annotations-bbox.csv
!rm /tmp/dataset/test-annotations-bbox.csv
!rm /tmp/dataset/validation-annotations-bbox.csv
!rm /tmp/dataset/class-descriptions-boxable.csv

!ls /tmp/dataset/

new-class-descriptions-boxable.csv  new-train-annotations-bbox.csv
new-test-annotations-bbox.csv	    new-validation-annotations-bbox.csv


In [82]:
train_annotations = pd.read_csv('/tmp/dataset/new-train-annotations-bbox.csv')
train_annotations.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000004f4400f6ec5,xclick,/m/04hgtk,1,0.300625,0.508125,0.224090,0.608777,1,0,0,0,0
1,000004f4400f6ec5,xclick,/m/04hgtk,1,0.544375,0.692500,0.179272,0.469655,0,0,0,0,0
2,000004f4400f6ec5,xclick,/m/04hgtk,1,0.768125,0.999375,0.648926,0.999066,0,1,0,0,0
3,000004f4400f6ec5,xclick,/m/0dzf4,1,0.143125,0.615625,0.597572,0.999066,1,1,0,0,0
4,000004f4400f6ec5,xclick,/m/0dzf4,1,0.403750,0.716250,0.478992,0.892624,1,0,0,0,0


In [83]:
print("Current size:", len(train_annotations))

train_annotations = train_annotations[ train_annotations.IsDepiction == 0 ]
train_annotations = train_annotations[ train_annotations.IsInside == 0 ]

print("New size:", len(train_annotations))

Current size: 734882
New size: 678042


In [84]:
train_annotations.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000004f4400f6ec5,xclick,/m/04hgtk,1,0.300625,0.508125,0.224090,0.608777,1,0,0,0,0
1,000004f4400f6ec5,xclick,/m/04hgtk,1,0.544375,0.692500,0.179272,0.469655,0,0,0,0,0
2,000004f4400f6ec5,xclick,/m/04hgtk,1,0.768125,0.999375,0.648926,0.999066,0,1,0,0,0
3,000004f4400f6ec5,xclick,/m/0dzf4,1,0.143125,0.615625,0.597572,0.999066,1,1,0,0,0
4,000004f4400f6ec5,xclick,/m/0dzf4,1,0.403750,0.716250,0.478992,0.892624,1,0,0,0,0


### Occluded

In [16]:
len(train_annotations[ train_annotations.IsOccluded == 1 ])

472020

### Truncated

In [17]:
len(train_annotations[ train_annotations.IsTruncated == 1 ])

126968

### GroupOf

In [18]:
len(train_annotations[ train_annotations.IsGroupOf == 1 ])

8325

### Nothing

In [19]:
len(train_annotations) - len(train_annotations[ train_annotations.IsOccluded == 1 ]) - len(train_annotations[ train_annotations.IsTruncated == 1 ]) - len(train_annotations[ train_annotations.IsGroupOf == 1 ])

70729

In [66]:
class_labels_dict = { row['label']:row['name'] for index, row in class_labels.iterrows()}
class_labels_dict
labels_class = { row['name']:row['label'] for index, row in class_labels.iterrows()}
labels_class

{'Human body': '/m/02p0tk3',
 'Human foot': '/m/031n1',
 'Human leg': '/m/035r7c',
 'Human head': '/m/04hgtk',
 'Human arm': '/m/0dzf4',
 'Human hand': '/m/0k65p'}

In [67]:
train_annotations.replace({'LabelName' : class_labels_dict}).groupby('LabelName').sum()

,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
LabelName,,,,,,,,,,
Human arm,194495,85229.359767,108833.839525,87665.511768,127390.685102,181885,31816,425,0,0
Human body,166472,64403.111384,102168.013673,58483.159714,126433.771994,86955,47508,5549,0,0
Human foot,2189,903.532892,1317.681044,1099.623627,1620.064162,1345,317,5,0,0
Human hand,71356,31955.878986,38965.590223,37422.364734,45600.325481,49172,8137,14,0,0
Human head,173875,76105.369504,96991.360930,58662.986931,87589.093671,101886,23044,1949,0,0
Human leg,69655,30647.222386,39081.318953,38303.032020,54633.141161,50777,16146,383,0,0


In [68]:
df_foot = train_annotations[train_annotations.LabelName == labels_class['Human foot']]
df_body = train_annotations[train_annotations.LabelName == labels_class['Human body']]
df_arm = train_annotations[train_annotations.LabelName == labels_class['Human arm']]
df_hand = train_annotations[train_annotations.LabelName == labels_class['Human hand']]
df_head = train_annotations[train_annotations.LabelName == labels_class['Human head']]
df_leg = train_annotations[train_annotations.LabelName == labels_class['Human leg']]
df_foot.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
42,00007902a7b96e8c,xclick,/m/031n1,1,0.243542,0.434502,0.805204,0.902250,0,0,0,0,0
83,0000e2205e460318,xclick,/m/031n1,1,0.057500,0.089375,0.621951,0.651032,1,0,0,0,0
84,0000e2205e460318,xclick,/m/031n1,1,0.070000,0.111250,0.632270,0.654784,1,0,0,0,0
85,0000e2205e460318,xclick,/m/031n1,1,0.144375,0.188125,0.645403,0.678236,1,0,0,0,0
86,0000e2205e460318,xclick,/m/031n1,1,0.166250,0.208125,0.652908,0.681989,1,0,0,0,0


In [71]:
# Filtering datasets
df_body = df_body.sample(len(df_foot))
df_arm = df_arm.sample(len(df_foot))
df_hand = df_hand.sample(len(df_foot))
df_head = df_head.sample(len(df_foot))
df_leg = df_leg.sample(len(df_foot))

In [75]:
print('df_body:', df_body.sum() )
print('df_arm:', df_arm.sum() )
print('df_hand:', df_hand.sum() )
print('df_head:', df_head.sum() )
print('df_leg:', df_leg.sum() )

df_body: ImageID        1951b374029fec0417b4123731480ad3b236b0d066f176...
Source         activemilxclickxclickxclickxclickxclickxclickx...
LabelName      /m/02p0tk3/m/02p0tk3/m/02p0tk3/m/02p0tk3/m/02p...
Confidence                                                  2189
XMin                                                     849.776
XMax                                                     1356.46
YMin                                                     764.221
YMax                                                     1671.94
IsOccluded                                                  1122
IsTruncated                                                  618
IsGroupOf                                                     90
IsDepiction                                                    0
IsInside                                                       0
dtype: object
df_arm: ImageID        dcf1b01245d05fdc2492e48cc08ca4d8c3335db02abd2d...
Source         xclickxclickxclickxclickxclickxclickxclickxc

In [76]:
final_dataset = df_foot
final_dataset = final_dataset.append([df_body,df_arm,df_hand,df_head,df_leg])
final_dataset.sum()

ImageID        00007902a7b96e8c0000e2205e4603180000e2205e4603...
Source         xclickxclickxclickxclickxclickxclickxclickxcli...
LabelName      /m/031n1/m/031n1/m/031n1/m/031n1/m/031n1/m/031...
Confidence                                                 13134
XMin                                                     5629.88
XMax                                                     7558.22
YMin                                                     5955.13
YMax                                                     8938.35
IsOccluded                                                  8890
IsTruncated                                                 2355
IsGroupOf                                                    145
IsDepiction                                                    0
IsInside                                                       0
dtype: object

In [78]:
final_dataset.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
42,00007902a7b96e8c,xclick,/m/031n1,1,0.243542,0.434502,0.805204,0.902250,0,0,0,0,0
83,0000e2205e460318,xclick,/m/031n1,1,0.057500,0.089375,0.621951,0.651032,1,0,0,0,0
84,0000e2205e460318,xclick,/m/031n1,1,0.070000,0.111250,0.632270,0.654784,1,0,0,0,0
85,0000e2205e460318,xclick,/m/031n1,1,0.144375,0.188125,0.645403,0.678236,1,0,0,0,0
86,0000e2205e460318,xclick,/m/031n1,1,0.166250,0.208125,0.652908,0.681989,1,0,0,0,0


In [85]:
final_dataset.to_csv('../data/train-annotations-bbox.csv', header=False, index = False)